In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, *-*"

Installed Packages Microsoft.SemanticKernel, 1.1.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.1.0-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;


In [3]:
#!import ../Utils/Settings.cs
#!import ../../../Plugins/CustomPlugin/CompanySearchPlugin.cs

In [4]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [5]:
Kernel kernel = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion("Your Azure OpenAI Service gpt-35-turbo-16k Deployment Name", Settings.AOAIEndpoint, Settings.AOAIKey)
            .Build();

In [6]:
var companySearchPluginObj = new CompanySearchPlugin();
var companySearchPlugin = kernel.ImportPluginFromObject(companySearchPluginObj, "CompanySearchPlugin");

In [7]:
var pluginDirectory = Path.Combine("../../..", "plugins");

In [8]:
var writetPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "WriterPlugin"));
var emailPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "EmailPlugin"));
var translatePlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "TranslatePlugin"));

In [9]:
string goal =  """
Check the weather in Guangzhou, use spanish to write emails abount dressing tips based on the results
""";

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [11]:
#pragma warning disable SKEXP0060

var originalPlan = await planner.CreatePlanAsync(kernel, goal);

In [12]:
Console.WriteLine(originalPlan);

{{!-- Step 1: Identify key values --}}
{{set "city" "Guangzhou"}}
{{set "language" "spanish"}}

{{!-- Step 2: Get weather information --}}
{{set "weatherResult" (CompanySearchPlugin-WeatherSearch city=city)}}

{{!-- Step 3: Get dressing tips --}}
{{set "dressingTips" (WriterPlugin-Tips input=weatherResult language=language)}}

{{!-- Step 4: Write an email --}}
{{set "emailContent" (EmailPlugin-WeatherMail input=dressingTips)}}

{{!-- Step 5: Output the final result --}}
{{json emailContent}}


In [13]:
originalPlan.ToString()

{{!-- Step 1: Identify key values --}}
{{set "city" "Guangzhou"}}
{{set "language" "spanish"}}

{{!-- Step 2: Get weather information --}}
{{set "weatherResult" (CompanySearchPlugin-WeatherSearch city=city)}}

{{!-- Step 3: Get dressing tips --}}
{{set "dressingTips" (WriterPlugin-Tips input=weatherResult language=language)}}

{{!-- Step 4: Write an email --}}
{{set "emailContent" (EmailPlugin-WeatherMail input=dressingTips)}}

{{!-- Step 5: Output the final result --}}
{{json emailContent}}

In [14]:
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

In [15]:
Console.WriteLine(originalPlanResult);

Queridos compañeros de viaje,

Espero que estén bien. Solo quería recordarles que debemos estar atentos a las condiciones de viaje en todo momento, especialmente en cuanto al clima. Según los consejos de vestimenta para Guangzhou, actualmente está a 2 grados y hay lluvia, por lo que debemos asegurarnos de tomar algunas precauciones.

Es muy importante vestirse adecuadamente en este clima. Les sugiero seguir estas recomendaciones:

1. Usar capas: Comiencen con un suéter cálido como base y agreguen una chaqueta impermeable encima para mantenerse abrigados y protegidos de la lluvia.

2. Usar botas impermeables: Dado que está lloviendo, es fundamental usar calzado adecuado. Opten por unas botas impermeables que mantengan sus pies secos y protegidos del agua.

3. Elegir pantalones o faldas largas: Para mantener las piernas calientes, opten por pantalones o faldas largas. Estas prendas proporcionarán una capa adicional de aislamiento contra el frío.

4. No olvidar una bufanda y guantes: Aseg